In [108]:
# Import required libraries
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# dropdown options list
options = []
options.append({'label':'All Sites', 'value':'ALL'})
launch_sites = spacex_df['Launch Site'].unique().tolist()
for site in launch_sites:
    options.append({'label':site,'value':site})

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=options,
                                             placeholder='Select a Launch Site Here',
                                             value='ALL',
                                             searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                                min=0,
                                                max=10000,
                                                step=1000,
                                                marks=dict([(i,str(i)) for i in range(0,10001,1000)]),
                                                value=[min(spacex_df['Payload Mass (kg)']), max(spacex_df['Payload Mass (kg)'])]
                                               ),
                                

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        data = spacex_df[spacex_df['class']==1]['Launch Site'].value_counts().reset_index()
        fig = px.pie(data, values='count', names='Launch Site', title="Pie Chart of Successful Launch Sites")
        return fig
    else:
        data = spacex_df[spacex_df['Launch Site'] == entered_site]
        data = data['class'].value_counts().reset_index()
        fig = px.pie(data, values='count', names='class', color='class', color_discrete_map={1: 'green', 0: 'red'}, title="Pie Chart of Successful Launch Sites")
        return fig
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")]
             )
def get_scatter_plot(entered_site, payload_slider):
    if entered_site == 'ALL':
        data = spacex_df[(spacex_df['Payload Mass (kg)'] > payload_slider[0]) & (spacex_df['Payload Mass (kg)'] < payload_slider[1])]
        fig = px.scatter(data, x='Payload Mass (kg)', y='class', color='Booster Version Category')
        return fig
    else:
        data = spacex_df[(spacex_df['Payload Mass (kg)'] > payload_slider[0]) & (spacex_df['Payload Mass (kg)'] < payload_slider[1])]
        data = data[data['Launch Site'] == entered_site]
        fig = px.scatter(data, x='Payload Mass (kg)', y='class', color='Booster Version Category')
        return fig


# Run the app
if __name__ == '__main__':
    #app.run_server(jupyter_mode='external', port=8052)
    app.run_server(port=8053)